In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from os import listdir
from PIL import Image
from collections import defaultdict

# Data loading

In [2]:
IMAGE_PATH = '../data/images'
LABEL_PATH = '../data/annotations'

In [3]:
def number_of_classes():
    return len(listdir(LABEL_PATH))

In [4]:
def get_class_map():
    ret = {}

    i = 0
    for fname in listdir(LABEL_PATH):
        img_class, _ = fname.split('.')
        ret[img_class] = i
        i += 1

    return ret

In [5]:
def get_dataloader(bs=256, train_fr=.6, max_images_per_class=1e9):
    # mapping from class names to integers
    class_map = get_class_map()

    # create a dictionary to hold our label vectors
    n_classes = len(class_map.keys())
    img_to_class = defaultdict(lambda: np.zeros(n_classes))

    # another dictionary to hold the actual image data
    img_to_data = dict()
    
    # loop through all the annotations
    for fname in listdir(LABEL_PATH):
        img_class, _ = fname.split('.')
        print(f'Reading class: {img_class}')
        
        # open the annotation file
        with open(f'{LABEL_PATH}/{fname}', 'r') as fh:

            # get image ids from annotation file
            img_ids = fh.read().splitlines()

            # gather the images with labels
            i = 0
            for img_id in img_ids:
                img_path = f'{IMAGE_PATH}/im{img_id}.jpg'
                img = Image.open(img_path)

                # some images are black-and-white so convert to rgb
                img_rgb = img.convert('RGB')
                img_data = np.asarray(img_rgb)
                
                img_data = img_data.flatten()
                img_to_data[img_id] = img_data

                # get one-hot encoded vector of image classes
                img_classes = img_to_class[img_id]

                # add new class to image vector
                img_class_id = class_map[img_class]
                img_classes[img_class_id] = 1

                # store the updated vector back
                img_to_class[img_id] = img_classes

                if i >= max_images_per_class:
                    break

                i += 1

    # collect data to a single array
    X = []
    y = []
    for img_id in img_to_class.keys():
        X.append(img_to_data[img_id])
        y.append(img_to_class[img_id])
        
    X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, train_size=train_fr, random_state=42)
    X_test, X_valid, y_test, y_valid = train_test_split(X_tmp, y_tmp, train_size=.5, test_size=.5, random_state=42)
    
    train_dataloader = DataLoader(TensorDataset(
        torch.tensor(X_train, dtype=torch.float),
        torch.tensor(y_train, dtype=torch.float)),
        shuffle=True,
        batch_size=bs)           

    valid_dataloader = DataLoader(TensorDataset(
        torch.tensor(X_valid, dtype=torch.float),
        torch.tensor(y_valid, dtype=torch.float)),
        shuffle=True,
        batch_size=bs)

    test_dataloader = DataLoader(TensorDataset(
        torch.tensor(X_test, dtype=torch.float),
        torch.tensor(y_test, dtype=torch.float)),
        shuffle=True,
        batch_size=bs)      

    return train_dataloader, valid_dataloader, test_dataloader

# Models

In [6]:
class TwoLayerModel(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_classes):
        super().__init__()
        self.bs = bs
        self.input_layer = nn.Linear(n_input, n_hidden1)
        self.hidden1 = nn.Linear(n_hidden1, n_hidden2)
        self.hidden2 = nn.Linear(n_hidden2, n_classes)
        self.relu = nn.ReLU()
        self.bn0 = nn.BatchNorm1d(n_input)
        self.bn1 = nn.BatchNorm1d(n_hidden1)
        self.bn2 = nn.BatchNorm1d(n_hidden2)

    def forward(self, x):
        x = self.bn0(x)
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.hidden2(x)

        return x

In [7]:
class OneLayerModel(nn.Module):
    def __init__(self, n_input, n_hidden, n_classes):
        super().__init__()

        self.input_layer = nn.Linear(n_input, n_hidden)
        self.hidden = nn.Linear(n_hidden, n_classes)
        self.relu = nn.ReLU()
        self.bn0 = nn.BatchNorm1d(n_input)
        self.bn1 = nn.BatchNorm1d(n_hidden)

    def forward(self, x):
        x = self.bn0(x)
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.hidden(x)

        return x

# Training and evaluation functions

In [8]:
def evaluate(dataloader, model, criterion, device, threshold=0.5):
    model.eval()

    f1_scores = []
    losses = []

    with torch.no_grad():
        for batch in dataloader:
            X, y = batch
            X = X.to(device)
            y = y.to(device)
            y_pred = model(X)

            loss = criterion(y_pred, y)
            losses.append(loss)

            with torch.no_grad():
                score = f1_score(y_pred.cpu() > threshold, y.cpu() == 1, average='micro')
                f1_scores.append(score)

    model.train()

    return torch.mean(torch.tensor(losses)), torch.mean(torch.tensor(f1_scores))

In [9]:
def train(train_dataloader, valid_dataloader, model, optimizer, criterion, device, n_epochs=50, verbose=True):
    model.train()

    if verbose:
        fmt = '{:<5} {:12} {:12} {:<9} {:<9}'
        print(fmt.format('Epoch', 'Train loss', 'Valid loss', 'Train F1', 'Valid F1'))

    for epoch in range(n_epochs):
        
        for i, batch in enumerate(train_dataloader):
            X, y = batch
            
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

        if verbose:
            train_loss, train_score = evaluate(train_dataloader, model, criterion, device)
            valid_loss, valid_score = evaluate(valid_dataloader, model, criterion, device)

            fmt = '{:<5} {:03.10f} {:03.10f} {:02.7f} {:02.7f}'
            print(fmt.format(epoch, train_loss, valid_loss, train_score, valid_score))

In [17]:
def visualize_predictions(model, device, dataloader, n_to_show=3, threshold=0.5):
    class_to_label = { v: k for k, v in get_class_map().items() }

    for i, batch in enumerate(dataloader):
        n_shown = 0
        
        X, y = batch
        X = X.to(device)

        y_pred = model(X) > threshold
        y = y == 1

        for i in range(len(y)):
            pred_classes = np.where(y_pred[i] == 1)[0]
            true_classes = np.where(y[i] == 1)[0]

            true_classes_str = ', '.join([class_to_label[i] for i in true_classes])
            pred_classes_str = ', '.join([class_to_label[i] for i in pred_classes])

            img = X[i].cpu().numpy().reshape((128, 128, 3)).astype(np.uint8)
            plt.title(f'True: {true_classes_str}, Predictions: {pred_classes_str}')
            plt.imshow(img)
            plt.pause(0.001)

            if n_shown >= n_to_show:
                return

            n_shown += 1
            

# Do the magic!

In [11]:
use_cuda = True

device = torch.device('cuda') if use_cuda else torch.device('cpu')

lr = 0.01
n_epochs = 10
bs = 256

n_classes = len(get_class_map().keys())

model = TwoLayerModel(128*128*3, 512, 256, n_classes).to(device)
#model = OneLayerModel(128*128*3, 128, n_classes).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

## Create and save dataloaders

In [12]:
if False:
    train_dataloader, valid_dataloader, test_dataloader = get_dataloader(bs=bs)
    torch.save(train_dataloader, '../data/train_dataloader.dat')
    torch.save(valid_dataloader, '../data/valid_dataloader.dat')
    torch.save(test_dataloader, '../data/test_dataloader.dat')

Reading class: river
Reading class: clouds
Reading class: baby
Reading class: car
Reading class: dog
Reading class: bird
Reading class: flower
Reading class: male
Reading class: portrait
Reading class: tree
Reading class: night
Reading class: female
Reading class: people
Reading class: sea


## Load dataloaders from disk

In [13]:
train_dataloader = torch.load('../data/train_dataloader.dat')
valid_dataloader = torch.load('../data/valid_dataloader.dat')
test_dataloader = torch.load('../data/test_dataloader.dat')

## Do the actual training

In [14]:
train(train_dataloader, valid_dataloader, model, optimizer, criterion, device, n_epochs=10)

Epoch Train loss   Valid loss   Train F1  Valid F1 
0     0.6367203593 0.6510396600 0.4245159 0.3464681
1     0.5925002694 0.6166226268 0.5013885 0.3631080
2     0.5545904040 0.5926333666 0.5563133 0.3555477
3     0.5031051636 0.5550322533 0.6764159 0.4277465
4     0.4596023262 0.5289166570 0.7188496 0.4104176
5     0.4126107991 0.5023339391 0.7481074 0.4021907
6     0.3665544689 0.4752071202 0.8073666 0.4274994
7     0.3171113133 0.4397108257 0.8359030 0.4337959
8     0.2763474584 0.4183203578 0.8541200 0.4516676
9     0.2584436834 0.4183796942 0.8490205 0.4384023


In [18]:
visualize_predictions(model, device, valid_dataloader, n_to_show=20)

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.